In [1]:
import psycopg2
import sys
import pprint
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import time , datetime , date

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as off

### Funciones de utilidad

In [2]:
pickle_storage = './dataframes/'
pickle_extension = '.pkl'

def get_pickle(dbname,name):
    filename = pickle_storage + dbname + '_'+ name + pickle_extension
    return pd.read_pickle(filename)

def put_pickle(dbname,name, frame):
    filename = pickle_storage  + dbname + '_'+ name + pickle_extension
    frame.to_pickle(filename)

### Cargamos el dataframe Joineado

In [3]:
df_joined = get_pickle('joined_1_second','btcusd_xrpbtc_xrpusd')

In [4]:
df_joined.head()

,btcusd_initial_ask_price,btcusd_initial_bid_price,btcusd_final_ask_price,btcusd_final_bid_price,xrpbtc_initial_ask_price,xrpbtc_initial_bid_price,xrpbtc_final_ask_price,xrpbtc_final_bid_price,xrpusd_initial_ask_price,xrpusd_initial_bid_price,xrpusd_final_ask_price,xrpusd_final_bid_price
start,,,,,,,,,,,,
2018-01-18 09:11:02,11712.0,11711.0,11712.0,11711.0,0.000132,0.000131,0.000132,0.000131,1.5338,1.5324,1.5338,1.5324
2018-01-18 09:11:03,11712.0,11711.0,11712.0,11709.0,0.000132,0.000131,0.000132,0.000131,1.5338,1.5324,1.5338,1.5324
2018-01-18 09:11:04,11712.0,11706.0,11709.0,11702.0,0.000132,0.000131,0.000132,0.000131,1.5338,1.5324,1.5337,1.5324
2018-01-18 09:11:05,11709.0,11699.0,11708.0,11699.0,0.000132,0.000131,0.000131,0.000131,1.5337,1.5302,1.5300,1.5296
2018-01-18 09:11:06,11708.0,11699.0,11707.0,11703.0,0.000131,0.000131,0.000131,0.000131,1.5300,1.5296,1.5300,1.5296


### Aplicamos el calculo de evaluacion de rentabilidad

#### Formula general del saldo generado: 
#### Qusd_final = Qusd( Qxrp( Qbitcoins(Qusd_inicial)))

Con
<li>Qbitcoins (Qusd_inicial) = (Qusd - Qusd x Cmm) / btcusd_BID</li>
<li>Qxrp (Qbitcoins) = (Qbitcoins - Qbitcoins x Cmt) / xrpbtc_ASK</li>
<li>Qusd (Qxrp) = (Qxrp - Qxrp x Cmt) / xrpusd_BID</li>

Para valores ctes de
<li>Qusd_inicial = 1000 ;</li> <li>Cmm = 0.001% ;</li> <li>Cmt = 0.002%</li>

Hacemos:
<li>Qusd_final = Qusd( Qxrp( Qbitcoins(Qusd_inicial)))</li>

Resultado:
<li>Si Qusd_final es mayor que Qusd_inicial -> arbitraje favorable</li>
<li>Si Qusd_final es menor o igual que Qusd_inicial, entonces arbitraje desfavorable</li>


#### Medir: 
<li>cantidad de arbitrjes favorables</li>
<li>ventana de tiempo que duran los mismos</li>


#### Funciones necesarias


In [20]:
def q_bitcoins():
    qusd = params['qusd_initial'] * (1-params['cmm'])
    column_name = 'btcusd_' + params['key'] + '_bid_price'
    btcusd_bid = params['dataframe'][column_name]
    return qusd / btcusd_bid

def q_xrp(q_bitcoins_):
    q_bitcoins_ = q_bitcoins_ * (1-params['cmt'])
    column_name = 'xrpbtc_' + params['key'] + '_ask_price'
    xrpbtc_ask = params['dataframe'][column_name]
    return q_bitcoins_ / xrpbtc_ask

def q_usd(q_xrp_):
    q_xrp_ = q_xrp_ * (1-params['cmt'])
    column_name = 'xrpusd_' + params['key'] + '_bid_price'
    xrpusd_bid = params['dataframe'][column_name]
    return q_xrp_ / xrpusd_bid

#### Probamos

In [30]:
params = {
    'cmm':0.001 ,
    'cmt':0.002,
    'qusd_initial': 1000 ,
    'key': 'initial' ,
    'dataframe' : df_joined.copy()
}

In [31]:
df_joined_result = df_joined.copy()

In [32]:
df_joined_result['q_usd_final'] = q_usd(q_xrp(q_bitcoins()))

In [38]:
df_joined_result['q_usd_profit'] =  df_joined_result['q_usd_final'] - params['qusd_initial'] 

In [39]:
print('Ganancias: ' , len(df_joined_result[df_joined_result.q_usd_profit > 0]))
print('Perdidas: ' , len(df_joined_result[df_joined_result.q_usd_profit < 0]))

Ganancias:  0
Perdidas:  467960


In [40]:
df_joined_result['q_usd_profit']

start
2018-01-18 09:11:02   -578.527160
2018-01-18 09:11:03   -578.463072
2018-01-18 09:11:04   -578.250955
2018-01-18 09:11:05   -577.359750
2018-01-18 09:11:06   -577.161813
2018-01-18 09:11:07   -577.195770
2018-01-18 09:11:08   -577.057481
2018-01-18 09:11:09   -576.960970
2018-01-18 09:11:10   -576.331305
2018-01-18 09:11:11   -576.220491
2018-01-18 09:11:12   -574.630568
2018-01-18 09:11:13   -574.618733
2018-01-18 09:11:14   -573.853812
2018-01-18 09:11:15   -574.524469
2018-01-18 09:11:16   -574.524469
2018-01-18 09:11:17   -574.560862
2018-01-18 09:11:30   -573.414832
2018-01-18 09:11:31   -574.331043
2018-01-18 09:11:32   -573.904870
2018-01-18 09:11:33   -573.583876
2018-01-18 09:11:34   -573.583876
2018-01-18 09:11:35   -573.583876
2018-01-18 09:11:36   -573.551350
2018-01-18 09:11:37   -573.268912
2018-01-18 09:11:39   -571.376231
2018-01-18 09:11:40   -571.449595
2018-01-18 09:11:41   -569.508946
2018-01-18 09:11:42   -569.476735
2018-01-18 09:11:43   -568.968172
2018-01-

In [28]:
df_joined_result['q_usd_final']

start
2018-01-18 09:11:02    421.536928
2018-01-18 09:11:03    421.640987
2018-01-18 09:11:04    421.925288
2018-01-18 09:11:05    422.838187
2018-01-18 09:11:06    422.693664
2018-01-18 09:11:07    422.804230
2018-01-18 09:11:08    422.762479
2018-01-18 09:11:09    423.632494
2018-01-18 09:11:10    423.668695
2018-01-18 09:11:11    424.859068
2018-01-18 09:11:12    425.344882
2018-01-18 09:11:13    426.146188
2018-01-18 09:11:14    425.475531
2018-01-18 09:11:15    425.475531
2018-01-18 09:11:16    425.439138
2018-01-18 09:11:17    426.641113
2018-01-18 09:11:30    425.668957
2018-01-18 09:11:31    426.095130
2018-01-18 09:11:32    425.884852
2018-01-18 09:11:33    426.416124
2018-01-18 09:11:34    426.416124
2018-01-18 09:11:35    426.448650
2018-01-18 09:11:36    427.011096
2018-01-18 09:11:37    428.156336
2018-01-18 09:11:39    428.550405
2018-01-18 09:11:40    428.513733
2018-01-18 09:11:41    430.095201
2018-01-18 09:11:42    430.994915
2018-01-18 09:11:43    430.357596
2018-01-

In [29]:
len(df_joined_result[df_joined_result.q_usd_final > 0])

467960